In [1]:
import os
os.environ["PYSPARK_PYTHON"] = r"C:\Users\ranjan\Desktop\spark-olist-pipeline\venv\Scripts\python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"] = r"C:\Users\ranjan\Desktop\spark-olist-pipeline\venv\Scripts\python.exe"
os.environ["spark.python.worker.reuse"] = "true"

In [2]:
import sys
sys.path.append(os.path.abspath("../src")) 
from bronze import create_spark_session, ingest_csv

spark = create_spark_session("OlistPipeline")

In [3]:
# Check AQE
print("Adaptive Query Execution Enabled:", spark.conf.get("spark.sql.adaptive.enabled"))

Adaptive Query Execution Enabled: true


In [4]:
# Define paths
base_input = "../data/"
base_output = "../../delta/bronze/"

In [5]:

# Bronze ingestion for all Olist tables
datasets = {
    "customers": "olist_customers_dataset.csv",
    "orders": "olist_orders_dataset.csv",
    "order_items": "olist_order_items_dataset.csv",
    "order_payments": "olist_order_payments_dataset.csv",
    "order_reviews": "olist_order_reviews_dataset.csv",
    "products": "olist_products_dataset.csv",
    "sellers": "olist_sellers_dataset.csv",
    "geolocation": "olist_geolocation_dataset.csv",
    "category_translation": "product_category_name_translation.csv"
}


In [ ]:
for name, filename in datasets.items():
    input_path = f"{base_input}{filename}"
    output_path = f"{base_output}{name}"

    print(f"\n[INFO] Ingesting {name} → {output_path}")
    
    if name == "orders":  # partition only orders
        ingest_csv(spark, input_path, output_path, partition_col="order_purchase_month", target_file_rows=50000)
    else:
        ingest_csv(spark, input_path, output_path, target_file_rows=50000)

    # Read back from Delta & show
    df = spark.read.format("delta").load(output_path)
    print(f"[INFO] Showing {name} table (first 5 rows):")
    df.show(5, truncate=False)




[INFO] Ingesting customers → ../../delta/bronze/customers


In [ ]:
print("Bronze layer  ingestion completed successfully!")

In [ ]:
silver_output = "../delta/silver/"


In [ ]:
# Curate Silver Layer
curate_sales(spark, bronze_base=bronze_output, silver_base=silver_output)

In [ ]:
# Verify Silver
print("\n[INFO] Silver Layer Sales Table:")
sales_df = spark.read.format("delta").load(f"{silver_output}/sales")
sales_df.show(5, truncate=False)

In [ ]:
sales_df.select("seller_id", "seller_salt").distinct().show(10)

In [ ]:
# Unique city/state samples for customers & sellers
silver_path = "../delta/silver/"
customers_df = spark.read.format("delta").load(silver_path + "customers")
sellers_df = spark.read.format("delta").load(silver_path + "sellers")

print("Customer city samples:", [row.customer_city for row in customers_df.select("customer_city").distinct().limit(5).collect()])
print("Customer state samples:", [row.customer_state for row in customers_df.select("customer_state").distinct().limit(5).collect()])

print("Seller city samples:", [row.seller_city for row in sellers_df.select("seller_city").distinct().limit(5).collect()])
print("Seller state samples:", [row.seller_state for row in sellers_df.select("seller_state").distinct().limit(5).collect()])

In [ ]:
print("Duplicate customers:", customers_df.count() - customers_df.dropDuplicates(["customer_id"]).count())
print("Duplicate sellers:", sellers_df.count() - sellers_df.dropDuplicates(["seller_id"]).count())

In [ ]:
sales_df = spark.read.format("delta").load(silver_path + "sales")

print("Sample seller_id_salted:")
sales_df.select("seller_salt").distinct().show(10, truncate=False)

In [ ]:
spark.stop()